In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import math

from kite_trade import KiteAPI

In [11]:
# AVVA
kiteAvva = KiteAPI("2pj8TcisHNvtWfkrekfcRnaNQqEO+A4wfwEDqei/SyBAuj6zwk0BMsR0HjkJgprE1IOWtsq6QB90ZB+spuNOHMv3JJMVKlrkgxDk7N6EbFNHmgqytVedFw==")

In [3]:
# ISHWAR
kiteIshwar = KiteAPI("yGJqY3pIUtp78N8otb2oLu8ShrVRNsGR/K9mSM8ifLuI9Vp+qv//mT4+rMyUvyGM3v28EQqKAkPom9+Z+I6hOtdElcSmFn4XgLVEt1d54+/MDGGXY1peZQ==")

In [4]:
# RAKESH
kiteRakesh = KiteAPI("G4Rh53SmcXaFRpCb6RcWiSkVkP2HHK0B4qPoYpSi3AN7A4JrtbmTd1fK4s/xSCH1eystouHs6Z0ED9QdEY79qMp4t4qMrWn2/JxiCsrF4LRiRcnCOf/6dw==")

In [12]:
kite = kiteAvva
holdings = pd.DataFrame(kite.holdings())
holdings

HTTPError: 400 Client Error: Bad Request for url: https://api.kite.trade/portfolio/holdings

In [5]:
# Function to fetch stock data and filter based on 20DMA
def filter_stocks_to_sell(holdings):
    portfolio_value_20DMA = 0
    portfolio_value_5DMA = 0
    portfolio_value = 0
    buffer = 0.975
    filtered_stocks = []
    for index, row in holdings.iterrows():
        try:
            # Fetch stock data
            symbol = row['tradingsymbol'].split('-')[0] 
            exchange = ".NS" if row['exchange'] == "NSE" else ".BO"
            stock_data = yf.download(symbol + exchange, start=pd.to_datetime('today') - pd.DateOffset(50),
                                      end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
            
            # Calculate 20DMA
            stock_data['20DMA'] = stock_data['Close'].rolling(window=20).mean()
            
            #####
            stock_data['20DMA'] = stock_data['Close'].rolling(window=20).mean()
            stock_data['5DMA'] = stock_data['Close'].rolling(window=5).mean()
            quantity = row['quantity'] + row['t1_quantity'] + row['collateral_quantity'] + row['used_quantity']
            ltp = row['last_price']
            dma_20 = quantity * stock_data['20DMA'].iloc[-1]
            dma_5 = quantity * stock_data['5DMA'].iloc[-1]
            
            if math.isnan(dma_20):
                dma_20 = ltp

            portfolio_value_20DMA = portfolio_value_20DMA + (dma_20 * buffer)
            portfolio_value_5DMA = portfolio_value_5DMA + (dma_5 * buffer)
            portfolio_value = portfolio_value + (quantity * ltp)
            
            # print(f"{symbol} :: {round(stock_data['20DMA'].iloc[-1])} :: {row['last_price']}")
            
            # Check if current price is less than 5DMA
            if row['last_price'] < (stock_data['5DMA'].iloc[-1] * buffer):
                filtered_stocks.append(symbol)
                print(f"{symbol} :: {round(stock_data['5DMA'].iloc[-1])}")
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            
    print(f"Portfolio Value after breaching 5DMA ::  {round(portfolio_value_5DMA)} ")
    print(f"Portfolio Value after breaching 20DMA ::  {round(portfolio_value_20DMA)} ")
    print(f"Current Porfolio Value :: {round(portfolio_value)}")
    return filtered_stocks

# Filter stocks based on 20DMA
stocks_to_sell = filter_stocks_to_sell(holdings)

5PAISA :: 553
Portfolio Value after breaching 5DMA ::  446176 
Portfolio Value after breaching 20DMA ::  433428 
Current Porfolio Value :: 463862


In [6]:
# Stocks to sell
stocks_to_sell = holdings[holdings['tradingsymbol'].isin(stocks_to_sell)]
stocks_to_sell

,tradingsymbol,exchange,instrument_token,isin,product,price,quantity,used_quantity,t1_quantity,realised_quantity,...,short_quantity,collateral_quantity,collateral_type,discrepancy,average_price,last_price,close_price,pnl,day_change,day_change_percentage
0,5PAISA,NSE,113921,INE618L01018,CNC,0,17,0,0,17,...,0,0,,False,582.9,536.35,575.3,-791.35,-38.95,-6.770381


In [ ]:
def execute_sell_order(stocks_to_sell):
    for index, row in stocks_to_sell.iterrows():
        if(row['quantity'] == 0):
            print(f"{row['tradingsymbol']} : Quantity 0 - Skipping")
            continue
        try:
            response = kite.place_order(
                variety=kite.VARIETY_REGULAR,
                exchange=row['exchange'],
                tradingsymbol=row['tradingsymbol'],
                transaction_type=kite.TRANSACTION_TYPE_SELL,
                quantity=row['quantity'],
                order_type=kite.ORDER_TYPE_MARKET,
                product=kite.PRODUCT_CNC,
                tag="BaliHedgeFund"
            )
            # Print the response
            print("Response:", response)
        except Exception as e:
            print(f"An error occurred while placing order for {row['tradingsymbol']}: {e}")


execute_sell_order(stocks_to_sell)

In [ ]:
# Function to extract and store data from a page
def extract_data(url, headers):
    response = requests.get(url, headers=headers)
    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='data-table')
        rows = table.find_all('tr')
        data_list = []
        for i, row in enumerate(rows[1:-1], start=1):  # Skipping header row
            if i % 16 != 0:  # Skip every 15th row after the first 15 rows
                cols = row.find_all('td')
                symbol = str(cols[2].find('a')).split('/')[2]
                data = [col.text.strip() for col in cols]
                data.append(symbol)
                data_list.append(data)
        return data_list
    return None

# URL of the website
base_url = 'https://www.screener.in/watchlist/5428524/'

# Session ID and CSRF token obtained from somewhere (e.g., through authentication)
session_id = 'gxja81spl0xn0hggiwu4g5kxmhnbv9mz'
csrf_token = '8uCAl5uBfrOlzD295ykL6XYQ6BNEPwQU'

# Construct headers with session ID and CSRF token
headers = {
    'Cookie': f'sessionid={session_id}',  # Assuming the session ID is passed via cookie
    'X-CSRFToken': csrf_token  # Assuming the CSRF token is passed in a header
}

try:
    data = extract_data(base_url, headers)
    if data:
        df = pd.DataFrame(data, columns=['S.No.', '', 'Name', 'Price', 'P/E', 'Cap', 'QtrSV', 'QtrPV', 'QoQS', 'QoQP', 
                                         'Prom.', 'DII', 'FII', 'ExpQV', 'RSI', 'MACD', 'PromC', 'SYMBOL'])
        print("Data extracted successfully.")
    else:
        print("No data found.")
except Exception as e:
    print(f"Failed to extract data. Error: {e}")

In [ ]:
# Function to fetch stock data and filter based on 20DMA
def filter_stocks_to_buy(df):
    lower_range = 1.01
    upper_range = 1.05
    filtered_stocks = []
    for index, row in df.iterrows():
        try:
            # Fetch stock data
            symbol = row['SYMBOL']
            stock_data = yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(50),
                                      end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
            
            # Calculate 20DMA
            stock_data['20DMA'] = stock_data['Close'].rolling(window=20).mean()
            
            # Check if current price is less than 20DMA
            if (stock_data['Close'].iloc[-1] > (stock_data['20DMA'].iloc[-1] * lower_range)) and (stock_data['Close'].iloc[-1] < (stock_data['20DMA'].iloc[-1] * upper_range)) :
                filtered_stocks.append((symbol, round(stock_data['Close'].iloc[-1])))
                print(f"{symbol} :: {round(stock_data['20DMA'].iloc[-1])}")
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    return filtered_stocks

# Filter stocks based on Greater than 20DMA
stocks_to_buy = filter_stocks_to_buy(df)

In [ ]:
stocks_to_buy

In [ ]:
def execute_buy_order(stocks_to_buy):
    for symbol in stocks_to_buy:
        try:
            response = kite.place_order(
                variety=kite.VARIETY_REGULAR,
                exchange=kite.EXCHANGE_NSE,
                tradingsymbol=symbol[0],
                transaction_type=kite.TRANSACTION_TYPE_BUY,
                quantity=round(10000/symbol[1]),
                order_type=kite.ORDER_TYPE_MARKET,
                product=kite.PRODUCT_CNC,
                tag="BaliHedgeFund"
            )
            # Print the response
            print("Response:", response)
        except Exception as e:
            print(f"An error occurred while placing order for {symbol[0]}: {e}")


execute_buy_order(stocks_to_buy)

## FnO Positions Sell Strategy

In [ ]:
positions = pd.DataFrame(kite.positions().get('net'))
fno_positions = positions[positions['exchange'] == kite.EXCHANGE_NFO]

In [ ]:
fno_positions

In [ ]:
# Function to fetch stock data and filter based on 20DMA
def filter_fno_to_sell(fno_positions):
    buffer = 1.01
    stock_data_list = []
    for index, row in fno_positions.iterrows():
        try:
            # Fetch stock data
            symbol = row['tradingsymbol'][:-8]
            exchange = ".NS" if row['exchange'] == "NFO" else ".BO"
            stock_data = yf.download(symbol + exchange, start=pd.to_datetime('today') - pd.DateOffset(50),
                                      end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
            
            # Calculate 5DMA
            stock_data['5DMA'] = stock_data['Close'].rolling(window=5).mean()
            
            print(f"{symbol} :: {round(stock_data['5DMA'].iloc[-1] * buffer)} :: {row['last_price']}")
            
            stock_info = {
                'SYMBOL': row['tradingsymbol'],
                'BUY_PRICE': row['average_price'],
                'LTP': row['last_price'],
                'QUANTITY': row['quantity'],
                '5DMA': round(stock_data['5DMA'].iloc[-1] * buffer)
            }
            
            # Add dictionary to the list
            stock_data_list.append(stock_info)
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    stock_data_df = pd.DataFrame(stock_data_list)
    return stock_data_df

# Filter stocks based on 5DMA
fno_positions_to_sell = filter_fno_to_sell(fno_positions)

In [ ]:
for index, row in fno_positions_to_sell.iterrows():
    if row['LTP'] < row['BUY_PRICE']:
        fno_positions_to_sell.loc[index, 'EXPOSURE'] = (row['BUY_PRICE'] * 0.02) * row['QUANTITY']
    elif row['5DMA'] < row['BUY_PRICE']:
        fno_positions_to_sell.loc[index, 'EXPOSURE'] = (2 * row['BUY_PRICE'] - row['LTP']) * 0.02  * row['QUANTITY']
    else:
        fno_positions_to_sell.loc[index, 'EXPOSURE'] = 0

In [ ]:
account_value = 1900000
percent_exposure = (fno_positions_to_sell['EXPOSURE'].sum() / account_value) * 100
round(percent_exposure,2)

In [ ]:
fno_positions_to_sell

## PORTFOLIO VALUE

In [4]:
combined_portfolio_value = 0
#pairs = [(kiteAvva, 'Avva'), (kiteIshwar, 'Ishwar'), (kiteRakesh, 'Rakesh')]
pairs = [(kiteAvva, 'Avva')]
#pairs = [(kiteAvva, 'Avva'), (kiteIshwar, 'Ishwar')]
for pair in pairs:
    net_cash = 0
    cash_used = 0
    positions_value = 0
    holding_value = 0
    positions_value_sell = 0
    positions_value_buy = 0
    
    kite = pair[0]
    name = pair[1]
    
    # Holding Value
    holdings = pd.DataFrame(kite.holdings())
    holdings['total_quantity'] = holdings['quantity'] + holdings['t1_quantity'] + holdings['collateral_quantity']
    holding_value = (holdings['total_quantity'] * holdings['last_price']).sum()
    print(f'Holding Value :: {round(holding_value)}')
    
    # Position Value
    positions = pd.DataFrame(kite.positions().get('net'))
    if not positions.empty:
        positions = positions[positions['exchange'].isin([kite.EXCHANGE_NSE, kite.EXCHANGE_BSE])]
        positions_value_sell = (positions['average_price'] * positions['sell_quantity']).sum() 
        positions_value_buy = (positions['last_price'] * positions['buy_quantity']).sum()
        positions_value = positions_value_sell + positions_value_buy
        print(f'Position Value :: {round(positions_value)}')
    
    # Remaining Cash
    margins = pd.DataFrame(kite.margins().get('equity'))
    opening_cash = margins.loc['cash']['available']
    cash_used = positions_value_buy
    net_cash = opening_cash - cash_used
    print(f'Cash Value :: {round(net_cash)}')
    
    # Total Portfolio Value
    total_portfolio_value = round(holding_value + positions_value + net_cash)
    print(f'Total Portfolio Value {name} ::  {total_portfolio_value}')
    print("\n")
    
    combined_portfolio_value = combined_portfolio_value + total_portfolio_value

print(f'Combined Portfolio Value ::  {combined_portfolio_value}')

HTTPError: 400 Client Error: Bad Request for url: https://api.kite.trade/portfolio/holdings

In [12]:
weekly_trades = ['MCDOWELL-N', 'ONGC', 'POWERGRID','TORNTPHARM', 'GAIL', 'PETRONET', 'BIOCON', 'CHAMBLFERT',
               'HAVELLS', 'CONCOR', 'NMDC', 'TATAPOWER', 'BEL', 'HAL','SUPREMEIND']

In [13]:
kite = kiteAvva
holdings = pd.DataFrame(kite.holdings())
weekly_trades = holdings[holdings['tradingsymbol'].isin(weekly_trades)][['tradingsymbol', 'pnl']]

In [14]:
weekly_trades

,tradingsymbol,pnl
1,BEL,-1279.200000
2,BIOCON,-23.899964
4,CHAMBLFERT,-2386.800000
5,CONCOR,-190.350000
9,HAVELLS,619.500000
12,MCDOWELL-N,-643.700000
13,NMDC,1930.300000
14,ONGC,545.600000
15,PETRONET,1336.500000
20,SUPREMEIND,1500.000000


In [15]:
weekly_trades['pnl'].sum()

3311.7500359999995